# BLAST Proteins

Execução do BLAST através do Biopython no ambiente jupyter.

In [11]:
from Bio.Blast.Applications import NcbimakeblastdbCommandline
from Bio.Blast.Applications import NcbiblastpCommandline

In [12]:
#Criação do banco
cLine = NcbimakeblastdbCommandline(dbtype="prot", title="goodProteins", input_file="goodProteins.fasta")

In [13]:
cLine

NcbimakeblastdbCommandline(cmd='makeblastdb', dbtype='prot', input_file='goodProteins.fasta', title='goodProteins')

In [14]:
print(cLine)

makeblastdb -dbtype prot -in goodProteins.fasta -title goodProteins


In [15]:
stdout, stderr = cLine()

In [19]:
# Execução do BLASTP
cLineProtein = NcbiblastpCommandline(query="goodProteins.fasta", db="goodProteins.fasta",
                                     evalue=1e-5, outfmt=6, out="resultado_tabular")

In [20]:
print(cLineProtein)

blastp -out resultado_tabular -outfmt 6 -query goodProteins.fasta -db goodProteins.fasta -evalue 1e-05


In [21]:
stdout, stderr = cLineProtein()

In [ ]:
# Execução do BLASTP (Formato CSV)
cLineProteinCSV = NcbiblastpCommandline(query="goodProteins.fasta", db="goodProteins.fasta",
                                     evalue=1e-5, outfmt=6, out="resultado.csv")

In [ ]:
stdout, stderr = cLineProteinCSV()

# Aplicação do Spark para leitura de arquivo tabular

In [22]:
from pyspark.sql import SparkSession

In [23]:
spark = SparkSession.builder.appName('blast').getOrCreate()

In [29]:
blastDF = spark.read.text("resultado_tabular")
#peopleDF.select("name", "age").write.format("parquet").save("namesAndAges.parquet")
blastDF

DataFrame[value: string]

In [31]:
blastDF.show()

+--------------------+
|               value|
+--------------------+
|sac|YPL071C	sac|Y...|
|sac|YLL050C	sac|Y...|
|sac|YLL050C	lei|C...|
|sac|YMR172W	sac|Y...|
|sac|YMR172W	sac|Y...|
|sac|YOR185C	sac|Y...|
|sac|YOR185C	sac|Y...|
|sac|YOR185C	lei|C...|
|sac|YOR185C	lei|C...|
|sac|YOR185C	sac|Y...|
|sac|YOR185C	lei|C...|
|sac|YOR185C	sac|Y...|
|sac|YOR185C	lei|C...|
|sac|YOR185C	lei|C...|
|sac|YOR185C	sac|Y...|
|sac|YOR185C	lei|C...|
|sac|YOR185C	sac|Y...|
|sac|YOR185C	lei|C...|
|sac|YOR185C	sac|Y...|
|sac|YOR185C	lei|C...|
+--------------------+
only showing top 20 rows



In [37]:
# Linhas com a Proteína sac|YOR185C

lines = blastDF.filter(blastDF.value.contains("sac|YOR185C"))
lines.collect()

[Row(value='sac|YOR185C\tsac|YOR185C\t100.000\t220\t0\t0\t1\t220\t1\t220\t8.56e-169\t462'),
 Row(value='sac|YOR185C\tsac|YLR293C\t97.727\t220\t4\t1\t1\t220\t1\t219\t1.07e-162\t447'),
 Row(value='sac|YOR185C\tlei|CAJ05272\t69.082\t207\t64\t0\t11\t217\t9\t215\t1.43e-107\t307'),
 Row(value='sac|YOR185C\tlei|CAJ02862\t35.185\t162\t102\t2\t13\t171\t7\t168\t1.15e-30\t111'),
 Row(value='sac|YOR185C\tsac|YOR089C\t34.118\t170\t106\t3\t8\t171\t2\t171\t5.26e-27\t102'),
 Row(value='sac|YOR185C\tlei|CBZ12137\t31.579\t190\t122\t4\t14\t195\t9\t198\t1.47e-26\t100'),
 Row(value='sac|YOR185C\tsac|YNL093W\t34.463\t177\t105\t4\t6\t171\t3\t179\t2.89e-26\t100'),
 Row(value='sac|YOR185C\tlei|CAJ08707\t34.211\t152\t98\t2\t14\t164\t33\t183\t9.19e-26\t99.4'),
 Row(value='sac|YOR185C\tlei|CAJ08729\t30.159\t189\t128\t3\t14\t198\t8\t196\t2.32e-24\t95.1'),
 Row(value='sac|YOR185C\tsac|YER031C\t32.468\t154\t101\t2\t14\t164\t14\t167\t5.05e-24\t94.7'),
 Row(value='sac|YOR185C\tlei|CAJ08532\t28.272\t191\t121\t5\t14\t19

In [36]:
# Quantidade de Linhas com a Proteína sac|YOR185C

blastDF.filter(blastDF.value.contains("sac|YOR185C")).count()

88